<h1>Numerical Methods in Julia</h1>
This Jupyter Notebook serves as supplementary material to the Julia code from the book [Numerical Methods for Scientific Computing](fff). By-and-large, the snippets are verbatim from the book with the an occasional explicit plot statement or variable declaration. These code snippets are minimal working toy algorithms meant to better understand the mathematics that goes into them. They are tools for tinkering and learning. Play with them and have fun. And, perhaps you can repurpose a few of them. The notebook is designed to be nonlinear⁠—you can jump around. It is assumed that the LinearAlgebra.jl and Plots.jl packages are available.

In [ ]:
using LinearAlgebra, Plots

<h1>Notebook Contents</h1>

 [Part 1: Numerical Linear Algebra](#label0)<br>
&emsp; [Chapter 1: A Review of Linear Algebra](#label1)<br>
&emsp; [Chapter 2: Direct Methods for Linear Systems](#label2)<br>
&emsp; [Chapter 3: Inconsistent Systems](#label3)<br>
&emsp; [Chapter 4: Computing Eigenvalues](#label4)<br>
&emsp; [Chapter 5: Iterative Methods for Linear Systems](#label5)<br>
&emsp; [Chapter 6: Fast Fourier Transform](#label6)<br>


<a name="label0"></a>
# Part 1: Numerical Linear Algebra
<a name="label1"></a>
## Chapter 1: A Review of Linear Algebra
**The Hilbert matrix.** The Hlbert matrix $\mathbf{H}$ is ill conditioned even for relatively small dimensions. Taking $\mathbf{H}^{-1}\mathbf{H}$ should give us the identity matrix. 

In [ ]:
hilbert(n)  = [1/(i+j-1) for i=1:n, j=1:n]

In [ ]:
using Images
[Gray.(1 .- abs.(hilbert(n)\hilbert(n))) for n ∈ (10,15,20,25,50)]

<a name="label2"></a>
## Chapter 2: Direct Methods for Linear Systems
**Gaussian elimination.** The following function implements a naïve Gaussian elimination algorithm for a matrix `A` and vector `b`. We'll verify the code using a random matrix-vector pair. 

In [ ]:
function gaussian_elimination(A,b)
  n = size(A,1)
  for j in 1:n
    A[j+1:n,j] /= A[j,j]
    A[j+1:n,j+1:n] -= A[j+1:n,j:j].*A[j:j,j+1:n]
  end
  for i in 2:n
    b[i:i] -= A[i:i,1:i-1]*b[1:i-1]
  end
  for i in n:-1:1
    b[i:i] = ( b[i] .- A[i:i,i+1:n]*b[i+1:n] )/A[i,i]
  end
  return b
end

In [ ]:
A = rand(8,8); b = rand(8,1)
[A\b gaussian_elimination(A,b)]

**Simplex method.** The following three functions (`get_pivot`, `row_reduce`, and `simplex`) implement a naïve simplex method

In [ ]:
function row_reduce(tableau,p)
   (i,j) = get_pivot(tableau)
   p[filter(x->x==i, p)] .= 0; p[j] = i 
   G = tableau[i:i,:]/tableau[i,j]
   tableau -= tableau[:,j:j]*G
   tableau[i,:] = G
  return(tableau,p)
end

In [ ]:
function get_pivot(tableau)
  j = argmax(tableau[end,1:end-1])
  a, b = tableau[1:end-1,j], tableau[1:end-1,end]
  k = findall(x-> x>0 , a)
  i = k[argmin(b[k]/a[k])]
  return(i,j) 
end

In [ ]:
function simplex(A,b,c)
  (m,n) = size(A)
  tableau = [[A I b] ; [c' zeros(1,m) 0]]
  p = zeros(Int32,1,m+n)
  while (any(tableau[end,1:end-1].>0))
    (tableau,p) = row_reduce(tableau,p)
  end
  x = [0;tableau[1:end-1,end]][p[1:n].+1]
  z = -tableau[end,end]
  return(z,x)
end

In [ ]:
unit(n,i) = (z=zeros(n,1);z[i]=1;z)
using SparseArrays
function revised_simplex(A,b,c)
 (m,n) = size(A)
  N = Vector(1:n); B = Vector(n .+ (1:m))
  A = [sparse(A) sparse(I, m, m)]
  ABinv = sparse(I, m, m)
  c = [c;zeros(m,1)]
  while(true)
    j = findfirst(x->x>0,(c[N]'.-(c[B]'*ABinv)*A[:,N])[:])
    if isnothing(j); break; end
    q = ABinv*A[:,N[j]]
    k = findall(x->x>0,q)
    i = k[argmin(ABinv[k,:]*b./q[k])]
    B[i], N[j] = N[j], B[i]
    ABinv -=  ((q - unit(m,i))/q[i])*ABinv[i:i,:]
  end
  i = findall(x -> x≤n, B)
  x = zeros(n,1)
  x[B[i]] = ABinv[i,:]*b
  z = c[1:n]'*x
  return(x,z)
end

<a name="label3"></a>
## Chapter 3: Inconsistent Systems
**Constrained least squares.** The constrained least squares problem of solving $\mathbf{Ax} = \mathbf{b}$ with the constraint condition $\mathbf{Cx}=\mathbf{d}$:

In [ ]:
function constrained_lstsq(A,b,C,d)
  x = [A'*A C'; C zeros(size(C,1),size(C,1))]\[A'*b;d]
  x[1:size(A,2)]
end

**Total least squares.**  The function `tls` solves the total least squares problem. We'll use it along with ordinary least squares to find Zipf's law coefficients for canon of Sherlock Holmes.} %

In [ ]:
function tls(A,B)
  n = size(A,2)
  V = svd([A B]).V
  -V[1:n,n+1:end]/V[n+1:end,n+1:end]
end

In [ ]:
using DelimitedFiles
bucket = "https://nmfsc.s3.amazonaws.com/"
T = readdlm(download(bucket*"sherlock.csv"), '\t')[:,2]
n = length(T)
A = [ones(n,1) log.(1:n)]
B = log.(T)
c₁ = A\B
c₂ = tls(A,B)
print("ordinary least squares:\n"*string(c₁)*"\n")
print("total least squares:\n"*string(c₂))

**Image compression** Let's use singular value decomposition to compress an image. We'll choose a nominal rank `k = 20` for demonstration. We'll use the Frobenius norm to compute the total pixelwise error in the compressed image. Then, we'll plot out all the singular values for comparison.

In [ ]:
using Images
bucket= "https://nmfsc.s3.amazonaws.com/"
A = Gray.(load(download(bucket*"laura.png")))
U, σ, V = svd(A);

In [ ]:
k = 20
Aₖ = Gray.( U[:,1:k] * Diagonal(σ[1:k]) * V[:,1:k]' )

In [ ]:
norm(A-Aₖ) ≈ norm(σ[k+1:end])

In [ ]:
ϵ² = 1 .- cumsum(σ)/sum(σ); scatter(ϵ²,xaxis=:log)

**Non-negative matrix factorization (NMF).** A naive implementation of non-negative matrix factorization using multiplicative updates (without a stopping criterion):

In [ ]:
function nmf(X,p=6)
  W = rand(Float64, (size(X,1), p))
  H = rand(Float64, (p,size(X,2)))
  for i in 1:50
    W = W.*(X*H')./(W*(H*H') .+ (W.≈0))
    H = H.*(W'*X)./((W'*W)*H .+ (H.≈0))
  end
  (W,H)
end

In [ ]:
using Images
bucket= "https://nmfsc.s3.amazonaws.com/"
A = Gray.(load(download(bucket*"laura.png")))
W,H = nmf(Float64.(A),20)
Gray.(W*H)

<a name="label4"></a>
## Chapter 4: Computing Eigenvalues
**Eigenvalue condition number.** The function `condeig` computes the eigenvalue condition number. Let's use it on a small random matrix.

In [ ]:
function condeig(A)
  Sᵣ = eigvecs(A)
  Sₗ = inv(Sᵣ')
  Sₗ ./= sqrt.(sum(abs.(Sₗ.^2), dims=1))
  1 ./ abs.(sum(Sᵣ.*Sₗ, dims=1))
end

In [ ]:
condeig(rand(4,4))

**PageRank.** The following minimal code computes the PageRank of the very  small graph by using the power method over 9 iterations <img src="https://nmfsc.s3.amazonaws.com/internet_graph.svg" alt="internet graph" title="internet graph" />

In [ ]:
H = [0 0 0 0 1; 1 0 0 0 0; 1 0 0 0 1; 1 0 1 0 0; 0 0 1 1 0]
v = all(x->x==0,H,dims=1)
H = H ./ (sum(H,dims=1)+v) 
n = size(H,1) 
d = 0.85
x = ones(n,1)/n
for i in 1:9
  x = d*(H*x) .+ d/n*(v*x)  .+ (1-d)/n
end 
x

<a name="label5"></a>
## Chapter 5: Iterative Methods for Linear Systems
 Let's set up a sparse matrix

In [ ]:
using SparseArrays
n = 50; x = (1:n)/(n+1); Δx = 1/(n+1) 
J = sparse(I, n, n)
D = spdiagm(-1 => ones(n-1), 0 => -2ones(n), 1 => ones(n-1) )
A = (kron(kron(D,J),J)+kron(J,kron(D,J))+kron(J,kron(J,D)))/Δx^2

<a name="label6"></a>
## Chapter 6: Fast Fourier Transform
**Radix-2 FFT.** This chapter introduces several naive functions. The radix-2 FFT algorithm is written as a recursive function `fftx2` and the inverse FFT is written as `ifftx2`.<a name="radix2fft"></a>

In [ ]:
function fftx2(c)
  n = length(c)
  ω = exp(-2im*π/n)
  if mod(n,2) == 0 
    k = collect(0:n/2-1)
    u = fftx2(c[1:2:n-1])
    v = (ω.^k).*fftx2(c[2:2:n]) 
    return([u+v; u-v])
  else 
    k = collect(0:n-1)
    F = ω.^(k*k')
    return(F*c)
  end 
end

In [ ]:
ifftx2(y) = conj(fftx2(conj(y)))/length(y);

**Fast Toeplitz multiplication.** The function `fasttoeplitz` computes the Toeplitz multiplication by padding out a Toeplitz matrix with zeros to make it circulant.

In [ ]:
function fasttoeplitz(c,r,x)
  n = length(x)
  Δ = nextpow(2,n) - n
  x₁ = [c; zeros(Δ); r[end:-1:2]]
  x₂ = [x; zeros(Δ+n-1)]
  ifftx2(fftx2(x₁).*fftx2(x₂))[1:n]
end

In [ ]:
using FFTW, Primes, Plots
N = 10000
smooth(n,N) = (1:N)[all.(x->x<=n,factor.(Set,1:N))]
t₁ = [(x = randn(n); (n,@elapsed(fft(x)))) for n∈primes(N)]
t₂ = [(x = randn(n); (n,@elapsed(fft(x)))) for n∈smooth(5,N)]
plot(t₁,label="prime"); plot!(t₂,label="5-smooth")
plot!(ylims=(0,t₁[end][2]*3))

In [ ]:
function bluestein(x)
  n = length(x)
  ω = exp.((1im*π/n)*(0:n-1).^2)
  ω.*fasttoeplitz(conj(ω),conj(ω),ω.*x)
end

In [ ]:
using Primes
function primitiveroot(n)
  ϕ = n - 1
  p = factor(Set,ϕ)
  for r = 2:n-1
    all([powermod(r, ϕ÷pᵢ, n) for pᵢ∈p].!=1) && return(r)
  end
end

In [ ]:
function rader_fft(x)
  n = length(x)
  r = primitiveroot(n)
  P₊ = powermod.(r, 0:n-2, n)
  P₋ = circshift(reverse(P₊),1)
  ω = exp.((2im*π/n)*P₋)
  c = x[1] .+ ifft(fft(ω).*fft(x[2:end][P₊]))
  [sum(x); c[reverse(invperm(P₋))]]    
end